In [ ]:
import mne
import numpy as np
import pickle
import torch
from pathlib import Path
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from sys import platform
import re

import mat73  # pip install mat73

## Load custom GSN HydroCel 129 montage from .sfp file

In [ ]:
gsn_montage_path = "./leadfield_calibrated/GSN_HydroCel_129.sfp"

# Read the montage (MNE will automatically scale to head_size=0.095m by default)
montage = mne.channels.read_custom_montage(
    gsn_montage_path,
    head_size=0.095,  # Default adult head size in meters
    coord_frame='head'
)

print(f"Loaded montage with {len(montage.ch_names)} channels")
print(f"Channel names: {montage.ch_names[:5]}...")  # Show first 5

In [ ]:
# Create channel info structure
info = mne.create_info(ch_names=montage.ch_names, sfreq=100.0, ch_types=['eeg'] * 129)
info.set_montage(montage)

print("\nChannel info created successfully")

In [ ]:
# Download and set up fsaverage template (best available for 9-10 years)
print("\nDownloading fsaverage template...")
from mne.datasets import fetch_fsaverage
fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = fs_dir.parent

# Set up paths for BEM and source space
subject = 'fsaverage'
trans = 'fsaverage'  # Use built-in fsaverage transformation
src = fs_dir / 'bem' / 'fsaverage-ico-5-src.fif'
bem = fs_dir / 'bem' / 'fsaverage-5120-5120-5120-bem-sol.fif'

In [ ]:
# Compute the forward solution (this creates the leadfield matrix)
print("\nComputing forward solution (this may take a few minutes)...")
fwd = mne.make_forward_solution(
    info, 
    trans=trans, 
    src=src, 
    bem=bem, 
    eeg=True,
    meg=False,
    mindist=5.0, # Minimum distance from inner skull (mm)
    n_jobs=-1, # Use all CPU cores
    verbose=True
)

print(f"\nForward solution computed successfully!")
print(f"Forward object: {fwd}")

In [ ]:
# Convert to fixed orientation and extract leadfield matrix
print("\nConverting to fixed orientation...")
fwd_fixed = mne.convert_forward_solution(
    fwd, 
    surf_ori=True, # Orient dipoles perpendicular to cortex
    force_fixed=True, # Use fixed orientation (reduces matrix size)
    use_cps=True
)

# Extract the leadfield matrix as numpy array
leadfield = fwd_fixed['sol']['data']
print(f"\nLeadfield matrix shape: {leadfield.shape}")
print(f"  - Channels: {leadfield.shape}")
print(f"  - Dipoles: {leadfield.shape}")

In [ ]:
# Convert to PyTorch tensor
leadfield_tensor = torch.from_numpy(leadfield).float().cpu()
print(f"\nPyTorch tensor created:")
print(f"  - Shape: {leadfield_tensor.shape}")
print(f"  - Dtype: {leadfield_tensor.dtype}")
print(f"  - Device: {leadfield_tensor.device}")

leadfield_tensor = torch.from_numpy(leadfield @ leadfield.T).float().cpu()
leadfield_tensor.shape

In [ ]:
# Save in multiple formats
output_dir = Path("./leadfield_calibrated")
output_dir.mkdir(exist_ok=True)

pt_path = output_dir / 'leadfield_129ch.pt'
torch.save(leadfield_tensor, pt_path)
print(f"Saved PyTorch tensor to: {pt_path}")

# Load Leadfield Matrix from GEDAI

In [ ]:
def _norm(lbl: str) -> str:
    return re.sub(r"[^0-9a-z]+", "", lbl.lower())

def load_mat_any(path):
    return mat73.loadmat(str(path))

def get_leadfield_selected(matdict, selected_channels):
    L = matdict["leadfield4GEDAI"]
    Gain = np.asarray(L["Gain"])                      # [channels x sources]
    gram = np.asarray(L["gram_matrix_avref"])         # [channels x channels]
    electrodes = L["electrodes"]

    # extract template labels
    if isinstance(electrodes, (list, tuple)):
        template_labels = [e["Name"] if isinstance(e, dict) else str(e) for e in electrodes]
    elif isinstance(electrodes, dict) and "Name" in electrodes:
        names = electrodes["Name"]
        template_labels = list(names) if isinstance(names, (list, tuple)) else [str(names)]
    else:
        template_labels = list(map(str, electrodes))

    lut = {_norm(t): i for i, t in enumerate(template_labels)}
    idx = np.array([lut.get(_norm(ch), -1) for ch in selected_channels], dtype=int)
    if (idx < 0).any():
        missing = [ch for ch, i in zip(selected_channels, idx) if i < 0]
        raise ValueError(f"Electrode labels not found in template: {missing}")

    Gain_sel = Gain[idx, :]
    gram_sel = gram[np.ix_(idx, idx)]
    labels_sel = [template_labels[i] for i in idx.tolist()]
    return Gain_sel, gram_sel, labels_sel

raw = mne.io.read_raw_eeglab("./samples/with_artifacts/artifact_jumps.set", preload=True)
eeg_labels = raw.copy().pick("eeg").ch_names  # preserves EEG order for picks='eeg'
M = load_mat_any("./leadfield_calibrated/fsavLEADFIELD_4_GEDAI.mat")
Gain_sel, gram_sel, labels_sel = get_leadfield_selected(M, eeg_labels)

np.save("./leadfield_calibrated/leadfield4GEDAI_eeg_61ch.npy", np.asarray(gram_sel, np.float64))

In [ ]:
raw = mne.io.read_raw_eeglab("./samples/with_artifacts/synthetic_bad_channels.set", preload=True)
eeg_labels = raw.copy().pick("eeg").ch_names  # preserves EEG order for picks='eeg'
M = load_mat_any("./leadfield_calibrated/fsavLEADFIELD_4_GEDAI.mat")
Gain_sel, gram_sel, labels_sel = get_leadfield_selected(M, eeg_labels)

np.save("./leadfield_calibrated/leadfield4GEDAI_eeg_67ch.npy", np.asarray(gram_sel, np.float64))

In [ ]:
raw = mne.io.read_raw_eeglab("./samples/with_artifacts/empirical_NOISE_EOG_EMG.set", preload=True)
eeg_labels = raw.copy().pick("eeg").ch_names  # preserves EEG order for picks='eeg'
M = load_mat_any("./leadfield_calibrated/fsavLEADFIELD_4_GEDAI.mat")
Gain_sel, gram_sel, labels_sel = get_leadfield_selected(M, eeg_labels)

np.save("./leadfield_calibrated/leadfield4GEDAI_eeg_27ch.npy", np.asarray(gram_sel, np.float64))